In [1]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from skopt import gp_minimize
from skopt.space import Integer
from numpy import mean
from skopt.utils import use_named_args
from skopt import gp_minimize
from skopt.space import Real
from sklearn.metrics import mean_squared_error
from skopt.space import Categorical
#
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score, median_absolute_error


In [ ]:
#dataset = pd.read_csv('D:/Code/Python/NCKH/A1/c.csv')
dataset = pd.read_csv('./Dataset/para.csv')
# 
features = ["Freq"]
targets = ["W", "L", "Lf"]
x = dataset[features]
y = dataset[targets]

In [3]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

# Scaling
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [4]:
search_space = [Categorical(categories=[True, False], name='fit_intercept')]

In [5]:
@use_named_args(search_space)
def evaluate_model(fit_intercept):
    model = LinearRegression(fit_intercept=fit_intercept)  # Create Linear Regression model with specified fit_intercept
    model.fit(x_train_scaled, y_train)
    y_pred = model.predict(x_test_scaled)
    error = mean_squared_error(y_test, y_pred)  # Calculate mean squared error
    return error

In [6]:
result = gp_minimize(evaluate_model, search_space)

C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [True] before, using random point [False]
  warnings.warn(
C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [True] before, using random point [True]
  warnings.warn(
C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\skopt\optimizer\optimizer.py:517: UserWarning: The objective has been evaluated at point [True] before, using random point [True]
  warnings.warn(
C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\skopt\

In [7]:
# Extract best parameters from optimization result
best_fit_intercept = result.x[0]

# Create the Linear Regression model with best parameters
best_model = LinearRegression(fit_intercept=best_fit_intercept)

# Train the model on the scaled training data
best_model.fit(x_train_scaled, y_train)


LinearRegression()

In [ ]:
new_dataset = pd.read_csv('./Dataset/para.csv')  # 
new_data = new_dataset["Freq"].values.reshape(-1, 1)  # 

# Scaling 
new_data_scaled = scaler.transform(new_data)

# 
y_pred = best_model.predict(x_test_scaled)


C:\Users\sachxn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
print('Best Accuracy: %.3f' % (1.0 - result.fun))
print('Best Parameters: fit_intercept=%s' % ('True' if result.x[0] else 'False'))

# 



Best Accuracy: -132.608
Best Parameters: fit_intercept=True


In [ ]:
FreqInput = float(input('Nhập vào tần số muốn dự đoán: '))

# 
new_data = [[FreqInput]]

# Scaling 
new_data_scaled = scaler.transform(new_data)

# 
y_pred = best_model.predict(new_data_scaled)

# 
print('Giá trị nhập vào là: ', FreqInput)
print('Giá trị dự đoán:', y_pred)

In [ ]:
def regression_report(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    explained_variance = explained_variance_score(y_true, y_pred)
    median_ae = median_absolute_error(y_true, y_pred)

    # Mean Bias Deviation (MBD)
    mbd = np.mean(y_pred - y_true)

    # Mean Absolute Percentage Error (MAPE)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    # Symmetric Mean Absolute Percentage Error (sMAPE)
    smape = np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred))) * 100

    report = pd.DataFrame({
        'Metric': [
            'Mean Absolute Error (MAE)',
            'Mean Squared Error (MSE)',
            'Root Mean Squared Error (RMSE)',
            'R² Score',
            'Explained Variance Score',
            'Median Absolute Error',
            'Mean Bias Deviation (MBD)',
            'Mean Absolute Percentage Error (MAPE)',
            'Symmetric Mean Absolute Percentage Error (sMAPE)'
        ],
        'Gausians': [mae, mse, rmse, r2, explained_variance, median_ae, mbd, mape, smape]
    })

    return report

report = regression_report(y_test, y_pred)
print(report)

                                             Metric    Gausians
0                         Mean Absolute Error (MAE)    8.560450
1                          Mean Squared Error (MSE)  133.608009
2                    Root Mean Squared Error (RMSE)   11.558893
3                                          R² Score    0.675289
4                          Explained Variance Score    0.678340
5                             Median Absolute Error    6.235167
6                         Mean Bias Deviation (MBD)   -1.072025
7             Mean Absolute Percentage Error (MAPE)   31.536181
8  Symmetric Mean Absolute Percentage Error (sMAPE)   39.147236
